<a href="https://colab.research.google.com/github/harryguiacorn/Trading-Analysis-Statistics/blob/main/Generate_1hr_Cloud_Signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance ta pandas

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=5050ea00fe63ea9bc7dafc2c9cbd8a45ffc69f07e0e1aa0c396f4fb369b2c248
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


# Download 1h data and generator cloud signals

In [3]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from ta.trend import IchimokuIndicator

class IchimokuAnalyzer:
    def __init__(self, symbol, period, interval, output_folder):
        self.symbol = symbol
        self.period = period
        self.interval = interval
        self.output_folder = output_folder

    def create_folder(self, folder_path):
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    def download_and_analyze_data(self):
        # Create folders for CSV and image files
        folder_path_csv = os.path.join(self.output_folder, 'csv', self.symbol)
        folder_path_img = os.path.join(self.output_folder, 'img', self.symbol)

        self.create_folder(folder_path_csv)
        self.create_folder(folder_path_img)

        # Fetch hourly data for the specified period
        data = yf.download(self.symbol, period=self.period, interval=self.interval)

        # Create an IchimokuIndicator instance
        ichimoku = IchimokuIndicator(data['High'], data['Low'])

        # Calculate Ichimoku values
        data['Ichimoku_A'] = ichimoku.ichimoku_a().shift(26)
        data['Ichimoku_B'] = ichimoku.ichimoku_b().shift(26)

        # Create the 'Cloud' column based on Ichimoku values
        data['Cloud'] = np.where(
            (data['Close'] > data['Ichimoku_A']) & (data['Close'] > data['Ichimoku_B']), 1,
            np.where((data['Close'] < data['Ichimoku_A']) & (data['Close'] < data['Ichimoku_B']), -1, 0)
        )

        # Export data to a CSV file
        csv_filename = os.path.join(folder_path_csv, f'{self.symbol}_hourly_cloud_data_{self.period}.csv')
        data.to_csv(csv_filename)
        print(f"Hourly data for {self.symbol} over the last {self.period} with Ichimoku cloud values exported to {csv_filename}")

if __name__ == "__main__":
    symbol = 'ES=F'
    period = '6mo'
    interval = '1h'
    output_folder = 'output-cloud-signal'

    analyzer = IchimokuAnalyzer(symbol, period, interval, output_folder)
    analyzer.download_and_analyze_data()


[*********************100%%**********************]  1 of 1 completed
Hourly data for ES=F over the last 6mo with Ichimoku cloud values exported to output/csv/ES=F/ES=F_hourly_cloud_data_6mo.csv
